In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=None)
n_samples = 1000
dataset = mnist_trainset.data.reshape(-1, 1, 28, 28)[:n_samples] / 255.

In [ ]:
print(dataset.shape)

In [ ]:
with h5py.File("../MRS_data/sample_data.h5") as hf:
    gt_fids = hf["ground_truth_fids"][()] #ground truth free induction decay signal value
    ppm = hf["ppm"][()] #ppm vector
    t = hf["t"][()] #time
gt_spec = np.fft.fftshift(np.fft.ifft(gt_fids,axis=1),axes=1)
gt_diff_spec = np.real(gt_spec[:,:,1]-gt_spec[:,:,0])

ppm = ppm[0]
t = t[0]

In [ ]:
transient = 2

In [ ]:
print(gt_diff_spec[0].shape)

In [ ]:
min_ppm = 2.5
max_ppm = 4
max_ind = np.amax(np.where(ppm >= min_ppm))
min_ind = np.amin(np.where(ppm <= max_ppm))

y_crop = gt_diff_spec[transient][min_ind:max_ind]
x_crop = gt_diff_spec[transient+1][min_ind:max_ind]
y_crop_norm = (y_crop-y_crop.min())/(y_crop.max()-y_crop.min())
x_crop_norm = (x_crop-x_crop.min())/(x_crop.max()-x_crop.min())

ppm_crop = ppm[min_ind:max_ind]

fig,ax = plt.subplots(1,2,figsize=(10,5))
mse = np.square(y_crop_norm-x_crop_norm).mean()
ax[0].plot(ppm,gt_diff_spec[transient],label="ground-truth1")
ax[0].plot(ppm,gt_diff_spec[transient+1],label="ground-truth2")
ax[0].invert_xaxis()
ax[0].set_xlabel("ppm")
ax[0].set_ylim(-1,1)
ax[0].vlines([min_ppm,max_ppm],-1,1,colors="red",linestyles="--")
ax[0].set_title("Ground truth spectrum")

print(f"MSE: {mse}")